In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
data = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/Titanic_Dataset/train.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
cols_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
data = data.drop(cols_drop, axis = 1)

In [5]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()

In [8]:
data['Sex'] = label_enc.fit_transform(data['Sex'])

In [9]:
data['Embarked'] = label_enc.fit_transform(data['Embarked'])

In [10]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [11]:
data['Age'].fillna(data['Age'].median(), inplace = True)

In [12]:
data.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [13]:
input_cols = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
output = ['Survived']

In [14]:
X_data = data[input_cols]
Y_data = data[output]

In [15]:
X_data.shape, Y_data.shape

((891, 7), (891, 1))

In [16]:
def entropy(col):
    favour = np.unique(col, return_counts = True)
    N = col.shape[0]
    ent = 0
    for i in favour[1]:
        prob = i/N
        ent += -1 * (prob * np.log2(prob))
    return ent

In [17]:
a = [0,1,0,0,1,0,1,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,1,1,1,1,1,1,0]
print(np.unique(a, return_counts = True))

(array([0, 1]), array([15, 14], dtype=int64))


In [18]:
b = np.unique(a, return_counts = True)
print(b[1])

[15 14]


In [19]:
def split_data(X_data, feat, value):
    X_right = pd.DataFrame([], columns = X_data.columns)
    X_left = pd.DataFrame([], columns = X_data.columns)
    
    for i in range(X_data.shape[0]):
        val = X_data[feat].loc[i]
        
        if val >= value:
            X_right = pd.concat([X_right, X_data.iloc[i:i + 1]])
        else:
            X_left = pd.concat([X_left, X_data.iloc[i:i + 1]])
    return X_left, X_right

In [20]:
left, right = split_data(data[:10], 'Pclass', 2)

In [21]:
left

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,1,0,38.0,1,0,71.2833,0
3,1,1,0,35.0,1,0,53.1000,2
6,0,1,1,54.0,0,0,51.8625,2


In [22]:
right

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
2,1,3,0,26.0,0,0,7.9250,2
4,0,3,1,35.0,0,0,8.0500,2
5,0,3,1,28.0,0,0,8.4583,1
7,0,3,1,2.0,3,1,21.0750,2
8,1,3,0,27.0,0,2,11.1333,2
9,1,2,0,14.0,1,0,30.0708,0


In [23]:
def info_gain(X_data, feat, value):
    left, right = split_data(X_data, feat, value)
    
    prob_l = left.shape[0] / X_data.shape[0]
    prob_r = right.shape[0]/ X_data.shape[0]
    
    i_gain = entropy(X_data.Survived) - (prob_l * entropy(left.Survived) +
                                        prob_r * entropy(right.Survived))
    return i_gain

In [24]:
for i in X_data.columns:
    print(i, info_gain(data, i, data[i].mean()))

Pclass 0.07579362743608165
Sex 0.2176601066606142
Age 0.0008836151229467681
SibSp 0.009584541813400071
Parch 0.015380754493137694
Fare 0.042140692838995464
Embarked 0.015909401384176403


In [25]:
class Decision_Trees:
    def __init__(self, depth = 0, max_depth = 5):
        self.left = None
        self.right = None
        self.feat = None
        self.value = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self, train_X):
        features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
        i_gain = []
        
        for i in features:
            i_gain.append(info_gain(train_X, i, train_X[i].mean()))
        
        self.feat = features[np.argmax(i_gain)]
        self.value = train_X[self.feat].mean()
        
        left, right = split_data(train_X, self.feat, self.value)
        left = left.reset_index(drop = True)
        right = right.reset_index(drop = True)
        print('Making Tree with Node:', self.feat, 'Depth', self.depth)
        
        if left.shape[0] == 0 or right.shape[0] == 0:
            if train_X.Survived.mean() >= 0.5:
                self.target = 'Survived'
            else:
                self.target = 'Not Survived'
            return 'Done Trainning'
        
        if self.depth >= self.max_depth:
            if train_X.Survived.mean() >= 0.5:
                self.target = 'Survived'
            else:
                self.target = 'Not Survived'
            return 'Done Trainning'
        
        self.left = Decision_Trees(depth = self.depth + 1)
        self.left.train(left)
        
        self.right = Decision_Trees(depth = self.depth + 1)
        self.right.train(right)
        
        if train_X.Survived.mean() >= 0.5:
            self.target = 'Survived'
        else:
            self.target = 'Not Survived'
        return 'Done Training'
    
    def predict(self, test_point):
        if test_point[self.feat] >= self.value:
            if self.right is None:
                return self.target
            return self.right.predict(test_point)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test_point)

In [26]:
split = 0.8

train_X = data[:int(split * data.shape[0])]
test_X = data[int(split * data.shape[0]):]

In [27]:
print(train_X.shape, test_X.shape)

(712, 8) (179, 8)


In [28]:
dt = Decision_Trees(max_depth = 7)
dt.train(train_X)

Making Tree with Node: Sex Depth 0
Making Tree with Node: Pclass Depth 1
Making Tree with Node: Embarked Depth 2
Making Tree with Node: Age Depth 3
Making Tree with Node: Pclass Depth 4
Making Tree with Node: Pclass Depth 5
Making Tree with Node: Pclass Depth 5
Making Tree with Node: Age Depth 4
Making Tree with Node: Pclass Depth 5
Making Tree with Node: SibSp Depth 5
Making Tree with Node: Age Depth 3
Making Tree with Node: SibSp Depth 4
Making Tree with Node: Age Depth 5
Making Tree with Node: SibSp Depth 5
Making Tree with Node: Pclass Depth 4
Making Tree with Node: Pclass Depth 5
Making Tree with Node: Age Depth 5
Making Tree with Node: Embarked Depth 2
Making Tree with Node: Fare Depth 3
Making Tree with Node: Parch Depth 4
Making Tree with Node: Embarked Depth 5
Making Tree with Node: Pclass Depth 5
Making Tree with Node: Fare Depth 4
Making Tree with Node: Fare Depth 5
Making Tree with Node: Pclass Depth 5
Making Tree with Node: Fare Depth 3
Making Tree with Node: Fare Depth 4


'Done Training'

In [29]:
print(dt.left.left.left.left.left.feat)
print(dt.left.left.left.left.right.feat)

Pclass
Pclass


In [30]:
from pprint import pprint
pprint(vars(dt.left.left.left.left.left))
pprint(vars(dt.left.left.left.right))

{'depth': 5,
 'feat': 'Pclass',
 'left': None,
 'max_depth': 5,
 'right': None,
 'target': 'Survived',
 'value': 1.0}
{'depth': 4,
 'feat': 'Age',
 'left': <__main__.Decision_Trees object at 0x00000181FE085A30>,
 'max_depth': 5,
 'right': <__main__.Decision_Trees object at 0x00000181FE08B160>,
 'target': 'Survived',
 'value': 45.63157894736842}


In [31]:
a = 10
b = 20

In [32]:
b
a

10

In [36]:
test_X = test_X.reset_index(drop = True)
pred = []

for i in range(test_X.shape[0]):
    output = dt.predict(test_X.loc[i])
    if output == 'Survived':
        pred.append(1)
    else:
        pred.append(0)

In [37]:
corr = 0

for i in range(test_X.shape[0]):
    if pred[i] == test_X.Survived[i]:
        corr += 1
print(corr/test_X.shape[0])

0.8603351955307262


In [38]:
from sklearn.tree import DecisionTreeClassifier

In [67]:
d_tree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)

In [68]:
d_tree.fit(X_data, Y_data)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [69]:
d_tree.score(test_X[input_cols], test_X.Survived)

0.9385474860335196

In [76]:
figure, _ = plt.subplots(ncols = 1, nrows = 1, figsize = (100, 100))
sklearn.tree.plot_tree(d_tree, filled = True)
plt.show()
figure.savefig('tree.png')